## Loading files

In [2]:
import pandas as pd

# 讀取 .pkl 文件
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

train_path ='output_1/train_data.pkl'
test_path ='output_1/test_data.pkl'
train_data = pd.read_pickle(train_path)
test_data = pd.read_pickle(test_path)

# 檢查數據

print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)


X_train.shape:  (1455563, 500)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 500)
y_test.shape:  (411972,)


### Count

In [3]:
from sklearn.preprocessing import LabelEncoder
import keras

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
# y_test = label_encode(label_encoder, y_test)

## Model 1

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger
import gc


# 定義模型
input_shape = X_train.shape[1]
output_shape = len(label_encoder.classes_)

model_input = Input(shape=(input_shape,))
X = model_input

#X_W1 = Dense(units=16)(X)  # 減少隱藏層單元數量
#H1 = ReLU()(X_W1)

H1_W2 = Dense(units=output_shape)(X)
H2 = Softmax()(H1_W2)

model_output = H2
model = Model(inputs=[model_input], outputs=[model_output])

# 使用混合精度編譯模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 訓練日誌
csv_logger = CSVLogger('logs/training_log.csv')

# 訓練
epochs = 50  # 減少訓練次數
batch_size = 8  # 減少批次大小

history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data=(X_test, y_test))

print('training finish')

2024-11-18 06:35:58.021262: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-18 06:35:58.021310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2024-11-18 06:36:00.307882: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1455563,8]
	 [[{{node Placeholder/_1}}]]
2024-11-18 06:36:00.308165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1455563,8]
	 [[{{node Placeholder/_1}}]]
2024-11-18 06:36:00.673372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-11-18 06:36:00.723351: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contai

 62574/181946 [=========>....................] - ETA: 12:14:02 - loss: 1.5060 - accuracy: 0.4531

### Model 2

In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Softmax
from keras.callbacks import CSVLogger

# 定義模型
input_shape = X_train.shape[1]
output_shape = len(label_encoder.classes_)

# Input layer
model_input = Input(shape=(input_shape,))
X = model_input

# 直接接輸出層並使用 Softmax
X_W1 = Dense(units=output_shape)(X)
model_output = Softmax()(X_W1)

# 建立模型
model = Model(inputs=[model_input], outputs=[model_output])

# 編譯模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 訓練日誌
csv_logger = CSVLogger('logs/training_log.csv')

# 訓練參數
epochs = 100
batch_size = 16

# 開始訓練
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data=(X_test, y_test))

print('training finish')

2024-11-18 06:35:41.178207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'X_train' is not defined

### TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 初始化 TfidfVectorizer，并限制特征数为 1000
tfidf_vect = TfidfVectorizer(max_features=1000)

# 使用 TF-IDF 向量化器进行训练集和测试集的向量化
train_tfidf = tfidf_vect.fit_transform(train_data["text"])
test_tfidf = tfidf_vect.transform(test_data["text"])

# 获取特征名称列表
feature_names = tfidf_vect.get_feature_names_out()

# 显示索引 100 到 110 的特征名称
selected_features = feature_names[100:110]
print("Features from index 100 to 110:", selected_features)

Features from index 100 to 110: ['asked' 'asking' 'ass' 'at' 'august' 'away' 'awesome' 'awkward' 'baby'
 'back']


In [2]:
import keras

y_train = train_data['emotion'] # 情感標籤
y_test = test_data['emotion']

## check dimension is a good habbit 確認數據正確性
print('X_train.shape: ', train_tfidf.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', test_tfidf.shape)
print('y_test.shape: ', y_test.shape)

2024-11-14 10:02:29.777587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'train_tfidf' is not defined